In [1]:
#CSV Bibliothek importieren 
import csv
#Pandas importieren und als pd bezeichnen
import pandas as pd
#Numpy importieren und als np bezeichnen 
import numpy as np
#Json Biliothek importieren
import json
#Request-Bibliothek importieren
import requests
#Python-Bibliothek für das URL-Laden.
import urlexpander

***

# Ladung der gekürzten URLs
### In diesem Notebook werden ausschließlich die Tweets von TweetsKB bearbeitet.  
1. Tweets werden in kleinen Teile aufgeteilt
2. Für jeden Teil werden die URLs geladen und gespeichert 

***

In [2]:
#Labels und Pfad für den TweetsKB

TweetsKB_labels    = ['Tweet Id', 'Benutzername', 'Zeitstempel', 'Abonnenten', 'Abonnierten', 'Retweets', 'Likes', 'Text', 'URLs', 'Ist_gelöscht' ]
TweetsKB_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsKB_stichprobe.tsv'

In [3]:
#Die tweets in TweetsKB werden gelesen

df_tweetskb = pd.read_csv(TweetsKB_Pfad, sep='\t', error_bad_lines=False, names=TweetsKB_labels)
print('Form :', df_tweetskb.shape)
df_tweetskb.head()

Form : (9409841, 10)


,Tweet Id,Benutzername,Zeitstempel,Abonnenten,Abonnierten,Retweets,Likes,Text,URLs,Ist_gelöscht
0,1178791636039589889,DCComics,2019-09-30 22:00:01+00:00,3349606,563,2,13,"Today on #DCDaily, the crew takes a deep dive ...",http://bit.ly/2mlNYpS:-:,False
1,1178791641265639424,seanberdyz,2019-09-30 22:00:02+00:00,1203,499,4,20,𝘪’𝘮 𝘨𝘰𝘯𝘯𝘢 𝘣𝘦 𝘳𝘪𝘨𝘩𝘵 𝘩𝘦𝘳𝘦 ⟶ https://t.co/kOxWQL5...,http://ourownside.carrd.co:-:,True
2,1178791640573579264,AnArtistAtBirth,2019-09-30 22:00:02+00:00,109,215,0,0,❛ 𝘠𝘰𝘶'𝘳𝘦 𝘴𝘪𝘵𝘵𝘪𝘯' 𝘰𝘯 𝘺𝘰𝘶𝘳 𝘧𝘦𝘦𝘭𝘪𝘯𝘨𝘴 . . . 𝘐'𝘮 𝘴𝘪...,null;,True
3,1178791644797255680,lagosboygang,2019-09-30 22:00:03+00:00,571,1401,0,0,@Tiffany_o_ @UG_derrick That's the one i wanna...,null;,True
4,1178791648056217600,Phenom_Hoops,2019-09-30 22:00:04+00:00,25630,1674,1,1,2022 6’9 Perry Smith Jr. showcasing his high-m...,https://www.phenomhoopreport.com/2022-69-perry...,False


In [8]:
#Nur die TweetId und URLs werden gespeichert
df_tweetskb_urls = df_tweetskb[['Tweet_Id', 'Urls' ]]

#Dataframe wird hier gespeichert.

pfad1 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/tweetskb_stichprobe_urls_1'

with open(pfad1, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    zaehler = 0
    for idx, row in df_tweetskb.iterrows():
        if zaehler < 392917:
            tsv_writer.writerow(row.values.tolist())
        else :
            break
        zaehler +=1
print('Zähler', zaehler)

Zähler 392917


***

In [8]:
#Die Methode get_URLs() bekommt eine Zeile von der Dataframe.
#In der Liste wird die TweetId gespeichert, 
#dann ladet sie in einer For-Schleife alle URLs und speichert sie wieder in der Liste.
#Die Liste wird direkt in einer tsv Datei gespeichert.



def get_Urls(url):
    urls_list = []
    l = []
    l3 =[]
    urls_list.append(url[0])
    if str(url[1]) != 'null;' and str(url[1]) != '0' :
        s = url[1].split(':-:')
        f = filter(None, s)
        l2 = list(f)
        for x in range(0, len(l2)):
            if urlexpander.is_short(l2[x]) :
                try:
                    l = urlexpander.expand(l2[x])
                except (requests.ConnectionError,
                        requests.exceptions.ReadTimeout,
                        requests.exceptions.Timeout,
                        requests.exceptions.ConnectTimeout,
                        requests.exceptions.SSLError,
                        requests.exceptions.TooManyRedirects,):
                    print("Timeout occurred")
            else :
                l = l2[x]
            l3.append(l)
    urls_list.append(l3)
    
    
    pfad2='C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/tweetskb_stichprobe_urls_loaded_1'
    with open(pfad2, 'a', newline='', encoding="utf-8") as out_file:
        tsv_writer = csv.writer(out_file, delimiter="\t")
        tsv_writer.writerow(urls_list)
    
    return urls_list


In [9]:
#Warnings-bibliothek importieren
import warnings

#Dadurch werden die Warnings in der Konsole gezeigt.
warnings.filterwarnings('ignore')

#Diese Methode liest jede Zeile von geteilten Tweets, und ruft die Funktion get_URLs().
pfad1='C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/tweetskb_stichprobe_urls_1'
x = 0
v =[]
with open(pfad1, 'r', encoding='utf-8') as tsv_file:
    read_tsv = csv.reader(tsv_file, delimiter="\t")
    for line in read_tsv:
        #l = urlexpander.expand(line[1])   

        #print(line)
        #print('Urls: ', get_Urls(line))
        v = get_Urls(line)
        #if x == 20:
        #    break
        x += 1
print('Zaehler', x)



Timeout occurred
Timeout occurred
Timeout occurred
Timeout occurred
Timeout occurred
Timeout occurred
Timeout occurred
Timeout occurred
Timeout occurred
Timeout occurred
Timeout occurred
Timeout occurred
Timeout occurred
Zaehler 392917
